In [130]:
!pip install numpy

Defaulting to user installation because normal site-packages is not writeable


In [131]:
import itertools
import numpy as np
import itertools

## Essentials

### Generating $F_q^k$ for a prime $q$.


In [132]:
def F(q,k):
    Fq=list(range(q))  # field F_q as a list [0, 1, 2, ..., q-1]
    a=list(itertools.product(*[Fq]*k))  #[Fq]*k is just k copies of Fq, * unpacks and itertools.product returns cartesian product 
    a2=[v for v in a if v.count(1)!=1 or v.count(0)!=k-1] #removes identity vectors
    id=[[0]*i+[1]+[0]*(k-i-1) for i in range(k)] #generates identity vectors
    ordered=id+a2 #adds identity at the beginning
    
    return [[e for e in v] for v in ordered] #writes square brackets

print(F(3, 4))

[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [0, 0, 0, 0], [0, 0, 0, 2], [0, 0, 1, 1], [0, 0, 1, 2], [0, 0, 2, 0], [0, 0, 2, 1], [0, 0, 2, 2], [0, 1, 0, 1], [0, 1, 0, 2], [0, 1, 1, 0], [0, 1, 1, 1], [0, 1, 1, 2], [0, 1, 2, 0], [0, 1, 2, 1], [0, 1, 2, 2], [0, 2, 0, 0], [0, 2, 0, 1], [0, 2, 0, 2], [0, 2, 1, 0], [0, 2, 1, 1], [0, 2, 1, 2], [0, 2, 2, 0], [0, 2, 2, 1], [0, 2, 2, 2], [1, 0, 0, 1], [1, 0, 0, 2], [1, 0, 1, 0], [1, 0, 1, 1], [1, 0, 1, 2], [1, 0, 2, 0], [1, 0, 2, 1], [1, 0, 2, 2], [1, 1, 0, 0], [1, 1, 0, 1], [1, 1, 0, 2], [1, 1, 1, 0], [1, 1, 1, 1], [1, 1, 1, 2], [1, 1, 2, 0], [1, 1, 2, 1], [1, 1, 2, 2], [1, 2, 0, 0], [1, 2, 0, 1], [1, 2, 0, 2], [1, 2, 1, 0], [1, 2, 1, 1], [1, 2, 1, 2], [1, 2, 2, 0], [1, 2, 2, 1], [1, 2, 2, 2], [2, 0, 0, 0], [2, 0, 0, 1], [2, 0, 0, 2], [2, 0, 1, 0], [2, 0, 1, 1], [2, 0, 1, 2], [2, 0, 2, 0], [2, 0, 2, 1], [2, 0, 2, 2], [2, 1, 0, 0], [2, 1, 0, 1], [2, 1, 0, 2], [2, 1, 1, 0], [2, 1, 1, 1], [2, 1, 1, 2], [2, 1, 2, 0], [2, 1, 2, 1], [2, 1

### Generating Im$(G)$

In [133]:
def image(M,q):
    K=[]
   # q=np.amax(M)+1 #could be that the matrix is over Fq but only has entries with values less than q-1 .... should change this
    k=len(M)
    for v in F(q,k):
        K.append(list(np.dot(v,M) %q)) #convert to a list so that "array" goes away
    return K

In [134]:
G=[[1, 0, 0, 1, 0],
        [0, 1, 0, 0, 1],
        [0, 0, 1, 0, 0]]
image(G,2)

[[1, 0, 0, 1, 0],
 [0, 1, 0, 0, 1],
 [0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 1, 1, 0, 1],
 [1, 0, 1, 1, 0],
 [1, 1, 0, 1, 1],
 [1, 1, 1, 1, 1]]

### Generating $\ker(H)$

In [135]:
def kernel(M,q):
    n=len(M)  
    k=len(M[0])
    K=[]
    for v in F(q,n):
        if np.array_equal(np.dot(v,M) %q, [0]*k):
            K.append(v)
    return K


## Hamming distance

### Computing Hamming distance between two vectors

In [136]:
def hdist(v1,v2,q):
    l=0
    for i in range(len(v1)):
        if not v1[i] %q==v2[i] % q:
            l+=1
    return l

hdist([0,2,1],[0,1,1],3)

1

### Computing the minimum distance of a code C

In [137]:
def mindist(C, q):
    k = len(C[0])
    D = [hdist(v,[0]*k, q) for v in C if v!=[0]*k]
    return min(D)

C=[[0,0,0],[1,0,1],[0,1,1],[1,1,0]]
mindist(C, 2)

2

In [138]:
C=[[1, 0, 0, 1, 0],
        [0, 1, 0, 0, 1],
        [0, 0, 1, 0, 0]]
mindist(C,2)

1

### Computing the maximum distance from any element in $F_q^n$ to some codeword.

In [139]:
def maxdist(C,q):
    k=len(C[0])
    A=F(q,k)
    D=[min([hdist(v,c,q) for c in C]) for v in A]
    return max(D)

maxdist(C,2)

3

In [140]:
maxdist(image([[1, 0, 1, 1],
               [0, 1, 1, 2]],3),3)

1

In [141]:
mindist(image([[1, 0, 1, 1],[0, 1, 1, 2]],3),3)

3

## Hamming codes

### Checking if two vectors are scalar multiples

In [142]:
def scalarmult(v1,v2,q):
    for a in range(1,q):
        if ([a*x % q for x in v1]) == [x %q for x in v2]: 
            return True
    return False

scalarmult([2,1],[1,2],3)

True

### Generating parity check matrices for Hamming codes

In [143]:
def hammingvectors(q,k):
    H = [(0,)*k]
    for v in F(q,k):
        for w in H:
            if scalarmult(v,w,q):
                break
        else:
            H.append(v)
    H.remove((0,)*k)
    return np.array(H) #np. turns list of vectors into a matrix
hammingvectors(3,4)

array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 1],
       [0, 0, 1, 2],
       [0, 1, 0, 1],
       [0, 1, 0, 2],
       [0, 1, 1, 0],
       [0, 1, 1, 1],
       [0, 1, 1, 2],
       [0, 1, 2, 0],
       [0, 1, 2, 1],
       [0, 1, 2, 2],
       [1, 0, 0, 1],
       [1, 0, 0, 2],
       [1, 0, 1, 0],
       [1, 0, 1, 1],
       [1, 0, 1, 2],
       [1, 0, 2, 0],
       [1, 0, 2, 1],
       [1, 0, 2, 2],
       [1, 1, 0, 0],
       [1, 1, 0, 1],
       [1, 1, 0, 2],
       [1, 1, 1, 0],
       [1, 1, 1, 1],
       [1, 1, 1, 2],
       [1, 1, 2, 0],
       [1, 1, 2, 1],
       [1, 1, 2, 2],
       [1, 2, 0, 0],
       [1, 2, 0, 1],
       [1, 2, 0, 2],
       [1, 2, 1, 0],
       [1, 2, 1, 1],
       [1, 2, 1, 2],
       [1, 2, 2, 0],
       [1, 2, 2, 1],
       [1, 2, 2, 2]])

### Showing that this hamming code is perfect.

In [144]:
C=kernel(hammingvectors(2,3),2)

In [145]:
mindist(C,2)

3

In [146]:
maxdist(C,2)

1

## Perfect codes

### Checking whether a code is perfect

In [147]:
def is_perfect(C,q):
    t=maxdist(C,q)
    d=mindist(C,q)
    if d==2*t+1:
        return True
    else:
        return False

In [148]:
is_perfect(C,2)

True

In [149]:
is_perfect(image([[1, 0, 0, 0, 0, 1, 1,1],
           [0, 1, 0, 0, 1, 0, 1,1],
           [0, 0, 1, 0, 1, 1, 0,1],
           [0, 0, 0, 1, 1, 1, 1,0]],2),2)

False

In [150]:
G=[     [1, 0, 0, 0, 0, 1, 1],
        [0, 1, 0, 0, 1, 0, 1],
        [0, 0, 1, 0, 1, 1, 0],
        [0, 0, 0, 1, 1, 1, 1]]
C=image(G,2)

In [151]:
is_perfect(C,2)

True

In [152]:
G=[     [1, 0,0,1],
        [0, 1, 1,1]]
        
C=image(G,2)
is_perfect(C,2)

False

In [153]:
H=[     [1, 1, 0],
        [1, 0, 1],
        [0, 1, 1],
        [1, 1, 1],
        [1, 0, 0],
        [0, 1, 0],
  [0, 0, 1]]
C=kernel(H,2)
is_perfect(C,2)

True

### Computing all generating matrices 

In [154]:
def genmatrices(q,k,n):
    A=F(q,k)
    S=itertools.product(A, repeat=n-k) #all possible subsets of A of length n-k. possibly repeated
    GENS=[]
    for V in S:
        G= [[0]*i+[1]+[0]*(k-i-1) for i in range(k)] #generates identity vectors of length k
        for v in V:
            G=np.column_stack((G, np.array(v)))
        GENS.append(G)
    return GENS

In [155]:
genmatrices(2,3,5)

[array([[1, 0, 0, 1, 1],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0]]),
 array([[1, 0, 0, 1, 0],
        [0, 1, 0, 0, 1],
        [0, 0, 1, 0, 0]]),
 array([[1, 0, 0, 1, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 1]]),
 array([[1, 0, 0, 1, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0]]),
 array([[1, 0, 0, 1, 0],
        [0, 1, 0, 0, 1],
        [0, 0, 1, 0, 1]]),
 array([[1, 0, 0, 1, 1],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 1]]),
 array([[1, 0, 0, 1, 1],
        [0, 1, 0, 0, 1],
        [0, 0, 1, 0, 0]]),
 array([[1, 0, 0, 1, 1],
        [0, 1, 0, 0, 1],
        [0, 0, 1, 0, 1]]),
 array([[1, 0, 0, 0, 1],
        [0, 1, 0, 1, 0],
        [0, 0, 1, 0, 0]]),
 array([[1, 0, 0, 0, 0],
        [0, 1, 0, 1, 1],
        [0, 0, 1, 0, 0]]),
 array([[1, 0, 0, 0, 0],
        [0, 1, 0, 1, 0],
        [0, 0, 1, 0, 1]]),
 array([[1, 0, 0, 0, 0],
        [0, 1, 0, 1, 0],
        [0, 0, 1, 0, 0]]),
 array([[1, 0, 0, 0, 0],
        [0, 1, 0, 1, 1],
        [0, 0, 1, 0, 1]]),

### Computing all matrices which generate perfect codes

In [156]:
def perfect(q,k,n):
    P=[]
    for G in genmatrices(q,k,n):
        if is_perfect(image(G,q),q):
            P.append(G)
    return P

In [157]:
P=perfect(2,4,7)
display(P)

[array([[1, 0, 0, 0, 0, 1, 1],
        [0, 1, 0, 0, 1, 0, 1],
        [0, 0, 1, 0, 1, 1, 0],
        [0, 0, 0, 1, 1, 1, 1]]),
 array([[1, 0, 0, 0, 0, 1, 1],
        [0, 1, 0, 0, 1, 0, 1],
        [0, 0, 1, 0, 1, 1, 1],
        [0, 0, 0, 1, 1, 1, 0]]),
 array([[1, 0, 0, 0, 0, 1, 1],
        [0, 1, 0, 0, 1, 1, 0],
        [0, 0, 1, 0, 1, 0, 1],
        [0, 0, 0, 1, 1, 1, 1]]),
 array([[1, 0, 0, 0, 0, 1, 1],
        [0, 1, 0, 0, 1, 1, 1],
        [0, 0, 1, 0, 1, 0, 1],
        [0, 0, 0, 1, 1, 1, 0]]),
 array([[1, 0, 0, 0, 0, 1, 1],
        [0, 1, 0, 0, 1, 1, 0],
        [0, 0, 1, 0, 1, 1, 1],
        [0, 0, 0, 1, 1, 0, 1]]),
 array([[1, 0, 0, 0, 0, 1, 1],
        [0, 1, 0, 0, 1, 1, 1],
        [0, 0, 1, 0, 1, 1, 0],
        [0, 0, 0, 1, 1, 0, 1]]),
 array([[1, 0, 0, 0, 1, 0, 1],
        [0, 1, 0, 0, 0, 1, 1],
        [0, 0, 1, 0, 1, 1, 0],
        [0, 0, 0, 1, 1, 1, 1]]),
 array([[1, 0, 0, 0, 1, 0, 1],
        [0, 1, 0, 0, 0, 1, 1],
        [0, 0, 1, 0, 1, 1, 1],
        [0, 0, 0, 1, 1, 1

The last three columns of each matrix have three 1s 
and one 0 and are pairwise distinc. 

In [158]:
perfect(3,2,4)

[array([[1, 0, 1, 1],
        [0, 1, 1, 2]]),
 array([[1, 0, 1, 2],
        [0, 1, 1, 1]]),
 array([[1, 0, 1, 1],
        [0, 1, 2, 1]]),
 array([[1, 0, 1, 2],
        [0, 1, 2, 2]]),
 array([[1, 0, 2, 1],
        [0, 1, 1, 1]]),
 array([[1, 0, 2, 2],
        [0, 1, 1, 2]]),
 array([[1, 0, 2, 1],
        [0, 1, 2, 2]]),
 array([[1, 0, 2, 2],
        [0, 1, 2, 1]])]

Up to scalar multiplication and reordering the rows of V where G=(id|V) we only have one matrix. 

In [159]:
for n in range(4,9):
    display(perfect(2,3,n))

[]

[]

[]

[]

[]

In [160]:
for n in range(5,7):
    display(perfect(2,4,n))

[]

[]

In [161]:
for n in range(6,8):
    display(perfect(2,5,n))

[]

[]

In [162]:
for k in range(1,4):
    display(perfect(2,k,4))

[]

[]

[]

### Generating one perfect matrix given $q,k$

In [163]:
def perfectone(q,k):
    for n in range(k+1,k+5):
        for G in genmatrices(q,k,n):
            if is_perfect(image(G,q),q):
                return G

In [164]:
perfectone(2,4)

array([[1, 0, 0, 0, 0, 1, 1],
       [0, 1, 0, 0, 1, 0, 1],
       [0, 0, 1, 0, 1, 1, 0],
       [0, 0, 0, 1, 1, 1, 1]])

## Encoding and decoding

### Introducing random errors 

In [165]:
import random
def error(w):
    q=max(w)+1
    n=len(w)
    # Choose a random index i
    i= random.randint(0,n-1)
    # Choose a random number a
    a= random.randint(0,q-1)
    w=list(w)
    w[i]=a #introducing an error in position i
    return w

In [166]:
error([0,1,0,1,0])

[0, 1, 0, 1, 0]

### Correcting a word given the code

In [167]:
def correct(C,w):
    q=max(w)+1
    
    m=min([hdist(w,v,q) for v in C])
    for v in C:
        if hdist(w,v,q)==m:
            return v

In [168]:
correct(image([[1, 0, 0, 0, 0, 1, 1],
        [0, 1, 0, 0, 1, 0, 1],
        [0, 0, 1, 0, 1, 1, 0],
        [0, 0, 0, 1, 1, 1, 1]],2),[0, 0, 0, 1, 0, 1, 0])

[0, 1, 0, 1, 0, 1, 0]

### Encoding

In [169]:
def encode(w):
    q=max(w)+1
    k=len(w)
    G=perfectone(q,k) 
    cw=np.dot(w, G) % q
    return cw

In [170]:
encode([0,1,0,1])

array([0, 1, 0, 1, 0, 1, 0])

In [171]:
error(encode([0,1,0,1]))

[0, 1, 1, 1, 0, 1, 0]

In [172]:
error(encode([0,1,1,1]))

[0, 1, 0, 1, 1, 0, 0]

### Breaking up texts into words

In [173]:
def breaking(text,k):
    brokentext=[text[i:i+k] for i in range(0,len(text),k)]
    return brokentext

In [174]:
breaking([1,2,2,4,4,2,1,2],2)

[[1, 2], [2, 4], [4, 2], [1, 2]]

### Decoding

In [175]:
def decode(cw,k):
    w=cw[:k]
    return w

In [176]:
decode([0,1,0,1,0,1,0],4)

[0, 1, 0, 1]

In [177]:
decode(correct(image(perfectone(2,4),2),[0,1,0,1,0,0,0]),4)

[0, 1, 0, 1]

### All together

In [178]:
def encodeco(k, text): #a complicated identity function
    words=breaking(text,k) #braking text into words of length k
    q = max(max(w) for w in words)+1
    G = perfectone(q,k) #finding a matrix generating a perfect code
    received_words=[]
    for w in words:
        cw = np.dot(w,G) % q  # encoding each word
        error(cw)  #introducing 1 error to each word. should compute t and introduce that many errors
        correct(image(G,q),cw) #correcting the errors
        dw=decode(cw,k) #removing redundancy
        received_words.append(dw)
    decodedtext=[a for w in received_words for a in w] #putting words back together
    print(decodedtext)

In [179]:
encodeco(4,[0,1,0,1,1,1,1,1]) 

[0, 1, 0, 1, 1, 1, 1, 1]
